In [3]:
import sys, os, time
import numpy as np
from datetime import datetime
from joblib import Parallel, delayed
import logging

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from estimators.statistical_descriptor import Nagler_WS
# from plot.figure_roc import ROC_plot
from utils.dataset_management import parse_pipeline
from utils.dataset_load import shuffle_data, DatasetLoader
from utils.fold_management import FoldManagement
from utils.label_management import LabelManagement
from utils.balance_management import *
from utils.figures import plot_boxplots, plot_roc_curves
from utils.files_management import *

In [102]:
x = {'commands': ["python evaluation/learning_models.py --data_path /home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5 --fold_method combinationFold --labeling_method crocus --balancing_method undersample --request (date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000)) --shuffle_data True --balance_data False --import_list from sklearn.svm import SVC --import_list from sklearn.neighbors import KNeighborsClassifier --import_list from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier --import_list from sklearn.linear_model import LogisticRegression --import_list from sklearn.neural_network import MLPClassifier --import_list from estimators.statistical_descriptor import * --import_list from estimators.band_transform import * --metrics_to_report f1_score_macro --metrics_to_report f1_score_weighted --metrics_to_report f1_score_multiclass --metrics_to_report accuracy_score --metrics_to_report precision_score_macro --metrics_to_report recall_score_macro --metrics_to_report roc_auc_score --metrics_to_report log_loss --metrics_to_report kappa_score --metrics_to_report confusion_matrix --seed 100150 --pipeline [['KNN_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['KNeighborsClassifier', {'n_neighbors': 50}]] --pipeline [['RandomForest_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['RandomForestClassifier', {'n_estimators': 200}, {'criterion': 'entropy'}]] --pipeline [['MLP_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['MLPClassifier', {'alpha': 0.01}]] --storage_path /home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/results/pipeline/run_26/group_0", "python evaluation/learning_models.py --data_path /home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5 --fold_method combinationFold --labeling_method crocus --balancing_method undersample --request (date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000)) --shuffle_data True --balance_data False --import_list from sklearn.svm import SVC --import_list from sklearn.neighbors import KNeighborsClassifier --import_list from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier --import_list from sklearn.linear_model import LogisticRegression --import_list from sklearn.neural_network import MLPClassifier --import_list from estimators.statistical_descriptor import * --import_list from estimators.band_transform import * --metrics_to_report f1_score_macro --metrics_to_report f1_score_weighted --metrics_to_report f1_score_multiclass --metrics_to_report accuracy_score --metrics_to_report precision_score_macro --metrics_to_report recall_score_macro --metrics_to_report roc_auc_score --metrics_to_report log_loss --metrics_to_report kappa_score --metrics_to_report confusion_matrix --seed 100150 --pipeline [['KNN_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['KNeighborsClassifier', {'n_neighbors': 50}]] --pipeline [['RandomForest_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['RandomForestClassifier', {'n_estimators': 200}, {'criterion': 'entropy'}]] --pipeline [['MLP_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['MLPClassifier', {'alpha': 0.01}]] --pipeline [['SVMrbf_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['SVC', {'kernel': 'rbf'}, {'probability': True}]] --pipeline [['LogisticR_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['LogisticRegression']] --pipeline [['AdaBoost_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['AdaBoostClassifier', {'n_estimators': 200}]] --storage_path /home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/results/pipeline/run_26/group_1"], 'commit_sha': '49de8221a85c92025f2382abf5134b3ee4a77c3c', 'end_time': [(2024, 7, 1, 13, 37, 16, 909068), (2024, 7, 1, 13, 41, 18, 717638)], 'executable': 'evaluation/learning_models.py', 'executable_command': 'python', 'experiment_id': 1, 'groups_of_parameters': [{'--balance_data': False, '--balancing_method': 'undersample', '--data_path': '/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5', '--fold_method': 'combinationFold', '--import_list': ['from sklearn.svm import SVC', 'from sklearn.neighbors import KNeighborsClassifier', 'from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier', 'from sklearn.linear_model import LogisticRegression', 'from sklearn.neural_network import MLPClassifier', 'from estimators.statistical_descriptor import *', 'from estimators.band_transform import *'], '--labeling_method': 'crocus', '--metrics_to_report': ['f1_score_macro', 'f1_score_weighted', 'f1_score_multiclass', 'accuracy_score', 'precision_score_macro', 'recall_score_macro', 'roc_auc_score', 'log_loss', 'kappa_score', 'confusion_matrix'], '--pipeline': [[['KNN_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['KNeighborsClassifier', {'n_neighbors': 50}]], [['RandomForest_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['RandomForestClassifier', {'n_estimators': 200}, {'criterion': 'entropy'}]], [['MLP_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['MLPClassifier', {'alpha': 0.01}]]], '--request': '(date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000))', '--seed': 100150, '--shuffle_data': True}, {'--balance_data': False, '--balancing_method': 'undersample', '--data_path': '/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5', '--fold_method': 'combinationFold', '--import_list': ['from sklearn.svm import SVC', 'from sklearn.neighbors import KNeighborsClassifier', 'from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier', 'from sklearn.linear_model import LogisticRegression', 'from sklearn.neural_network import MLPClassifier', 'from estimators.statistical_descriptor import *', 'from estimators.band_transform import *'], '--labeling_method': 'crocus', '--metrics_to_report': ['f1_score_macro', 'f1_score_weighted', 'f1_score_multiclass', 'accuracy_score', 'precision_score_macro', 'recall_score_macro', 'roc_auc_score', 'log_loss', 'kappa_score', 'confusion_matrix'], '--pipeline': [[['KNN_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['KNeighborsClassifier', {'n_neighbors': 50}]], [['RandomForest_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['RandomForestClassifier', {'n_estimators': 200}, {'criterion': 'entropy'}]], [['MLP_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['MLPClassifier', {'alpha': 0.01}]], [['SVMrbf_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['SVC', {'kernel': 'rbf'}, {'probability': True}]], [['LogisticR_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['LogisticRegression']], [['AdaBoost_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['AdaBoostClassifier', {'n_estimators': 200}]]], '--request': '(date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000))', '--seed': 100150, '--shuffle_data': True}], 'main_pid': 30846, 'pids': ['30860', '31348'], 'repertories': ['results/pipeline/run_26/group_0', 'results/pipeline/run_26/group_1'], 'run_id': 26, 'start_time': [(2024, 7, 1, 13, 34, 35, 443214), (2024, 7, 1, 13, 37, 16, 962683)], 'status': ['finished', 'finished'], 'storage_path': 'results/pipeline/run_26', 'working_directory': '/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline'}


In [81]:
[x["groups_of_parameters"][0]["--pipeline"][i][0][0] for i in range(len(x["groups_of_parameters"][0]["--pipeline"]))]

['KNN_direct', 'RandomForest_direct', 'MLP_direct']

{'commands': ["python evaluation/learning_models.py --data_path /home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5 --fold_method combinationFold --labeling_method crocus --balancing_method undersample --request (date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000)) --shuffle_data True --balance_data False --import_list from sklearn.svm import SVC --import_list from sklearn.neighbors import KNeighborsClassifier --import_list from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier --import_list from sklearn.linear_model import LogisticRegression --import_list from sklearn.neural_network import MLPClassifier --import_list from estimators.statistical_descriptor import * --import_list from estimators.band_transform import * --metrics_to_report f1_score_macro --metrics_to_report f1_score_weighted --metrics_to_report f1_score_multiclass --metrics_to_report accuracy_score --metrics_to_report precision_score_macro --metrics_to_report recall_score_macro --metrics_to_report roc_auc_score --metrics_to_report log_loss --metrics_to_report kappa_score --metrics_to_report confusion_matrix --seed 100150 --pipeline [['KNN_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['KNeighborsClassifier', {'n_neighbors': 50}]] --pipeline [['RandomForest_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['RandomForestClassifier', {'n_estimators': 200}, {'criterion': 'entropy'}]] --pipeline [['MLP_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['MLPClassifier', {'alpha': 0.01}]] --storage_path /home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/results/pipeline/run_26/group_0", "python evaluation/learning_models.py --data_path /home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5 --fold_method combinationFold --labeling_method crocus --balancing_method undersample --request (date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000)) --shuffle_data True --balance_data False --import_list from sklearn.svm import SVC --import_list from sklearn.neighbors import KNeighborsClassifier --import_list from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier --import_list from sklearn.linear_model import LogisticRegression --import_list from sklearn.neural_network import MLPClassifier --import_list from estimators.statistical_descriptor import * --import_list from estimators.band_transform import * --metrics_to_report f1_score_macro --metrics_to_report f1_score_weighted --metrics_to_report f1_score_multiclass --metrics_to_report accuracy_score --metrics_to_report precision_score_macro --metrics_to_report recall_score_macro --metrics_to_report roc_auc_score --metrics_to_report log_loss --metrics_to_report kappa_score --metrics_to_report confusion_matrix --seed 100150 --pipeline [['KNN_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['KNeighborsClassifier', {'n_neighbors': 50}]] --pipeline [['RandomForest_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['RandomForestClassifier', {'n_estimators': 200}, {'criterion': 'entropy'}]] --pipeline [['MLP_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['MLPClassifier', {'alpha': 0.01}]] --pipeline [['SVMrbf_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['SVC', {'kernel': 'rbf'}, {'probability': True}]] --pipeline [['LogisticR_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['LogisticRegression']] --pipeline [['AdaBoost_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['AdaBoostClassifier', {'n_estimators': 200}]] --storage_path /home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/results/pipeline/run_26/group_1"], 'commit_sha': '49de8221a85c92025f2382abf5134b3ee4a77c3c', 'end_time': [datetime.datetime(2024, 7, 1, 13, 37, 16, 909068), datetime.datetime(2024, 7, 1, 13, 41, 18, 717638)], 'executable': 'evaluation/learning_models.py', 'executable_command': 'python', 'experiment_id': 1, 'groups_of_parameters': [{'--balance_data': False, '--balancing_method': 'undersample', '--data_path': '/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5', '--fold_method': 'combinationFold', '--import_list': ['from sklearn.svm import SVC', 'from sklearn.neighbors import KNeighborsClassifier', 'from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier', 'from sklearn.linear_model import LogisticRegression', 'from sklearn.neural_network import MLPClassifier', 'from estimators.statistical_descriptor import *', 'from estimators.band_transform import *'], '--labeling_method': 'crocus', '--metrics_to_report': ['f1_score_macro', 'f1_score_weighted', 'f1_score_multiclass', 'accuracy_score', 'precision_score_macro', 'recall_score_macro', 'roc_auc_score', 'log_loss', 'kappa_score', 'confusion_matrix'], '--pipeline': [[['KNN_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['KNeighborsClassifier', {'n_neighbors': 50}]], [['RandomForest_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['RandomForestClassifier', {'n_estimators': 200}, {'criterion': 'entropy'}]], [['MLP_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['MLPClassifier', {'alpha': 0.01}]]], '--request': '(date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000))', '--seed': 100150, '--shuffle_data': True}, {'--balance_data': False, '--balancing_method': 'undersample', '--data_path': '/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5', '--fold_method': 'combinationFold', '--import_list': ['from sklearn.svm import SVC', 'from sklearn.neighbors import KNeighborsClassifier', 'from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier', 'from sklearn.linear_model import LogisticRegression', 'from sklearn.neural_network import MLPClassifier', 'from estimators.statistical_descriptor import *', 'from estimators.band_transform import *'], '--labeling_method': 'crocus', '--metrics_to_report': ['f1_score_macro', 'f1_score_weighted', 'f1_score_multiclass', 'accuracy_score', 'precision_score_macro', 'recall_score_macro', 'roc_auc_score', 'log_loss', 'kappa_score', 'confusion_matrix'], '--pipeline': [[['KNN_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['KNeighborsClassifier', {'n_neighbors': 50}]], [['RandomForest_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['RandomForestClassifier', {'n_estimators': 200}, {'criterion': 'entropy'}]], [['MLP_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['MLPClassifier', {'alpha': 0.01}]], [['SVMrbf_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['SVC', {'kernel': 'rbf'}, {'probability': True}]], [['LogisticR_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['LogisticRegression']], [['AdaBoost_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['AdaBoostClassifier', {'n_estimators': 200}]]], '--request': '(date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000))', '--seed': 100150, '--shuffle_data': True}], 'main_pid': 30846, 'pids': ['30860', '31348'], 'repertories': ['results/pipeline/run_26/group_0', 'results/pipeline/run_26/group_1'], 'run_id': 26, 'start_time': [datetime.datetime(2024, 7, 1, 13, 34, 35, 443214), datetime.datetime(2024, 7, 1, 13, 37, 16, 962683)], 'status': ['finished', 'finished'], 'storage_path': 'results/pipeline/run_26', 'working_directory': '/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline'}


In [82]:
[x["groups_of_parameters"][i]["--pipeline"][k][0][0] for k in range(len(x["groups_of_parameters"][0]["--pipeline"])) for i in range(len(x["groups_of_parameters"]))]

['KNN_direct',
 'KNN_direct',
 'RandomForest_direct',
 'RandomForest_direct',
 'MLP_direct',
 'MLP_direct']

In [83]:
def dump_h5(obj, filename):
    with h5py.File(filename, 'w') as h5file:
        pickled_obj = pickle.dumps(obj)
        h5file.create_dataset('data', data=np.void(pickled_obj))

def load_h5(filename):
    with h5py.File(filename, 'r') as h5file:
        pickled_obj = h5file['data'][()]
        return pickle.loads(pickled_obj)

In [84]:
dump_h5(x, "./tset.h5")

In [85]:
g0 = load_h5("/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/results/pipeline/run_25/group_0/models/KNN_direct/metrics.h5")
g1 = load_h5("/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/results/pipeline/run_25/group_1/models/KNN_direct/metrics.h5")

In [86]:
len(g0)

7

In [87]:
([41]+g1)

[41,
 {'f1_score_macro': 78.64,
  'f1_score_weighted': 79.86999999999999,
  'f1_score_multiclass': array([83.56, 73.71]),
  'accuracy_score': 79.77,
  'precision_score_macro': 78.38000000000001,
  'recall_score_macro': 78.97,
  'roc_auc_score': 78.97,
  'log_loss': 53.68000000000001,
  'kappa_score': 57.29,
  'confusion_matrix':          0        1
  0  82.1617  17.8383
  1  24.2291  75.7709,
  'y_true': array(['0', '0', '0', ..., '1', '1', '1'], dtype='<U21'),
  'y_pred': array([[0.48, 0.52],
         [0.54, 0.46],
         [0.98, 0.02],
         ...,
         [0.18, 0.82],
         [0.04, 0.96],
         [0.12, 0.88]]),
  'fold': 0,
  'training_time': 10.227454900741577,
  'prediction_time': 2.8009791374206543},
 {'f1_score_macro': 86.68,
  'f1_score_weighted': 86.95,
  'f1_score_multiclass': array([88.85, 84.51]),
  'accuracy_score': 87.03,
  'precision_score_macro': 87.29,
  'recall_score_macro': 86.33999999999999,
  'roc_auc_score': 86.33999999999999,
  'log_loss': 32.67,
  'kappa

In [105]:
x = {'commands': ["python evaluation/learning_models.py --data_path /home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5 --fold_method combinationFold --labeling_method crocus --balancing_method undersample --request (date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000)) --shuffle_data True --balance_data False --import_list from sklearn.svm import SVC --import_list from sklearn.neighbors import KNeighborsClassifier --import_list from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier --import_list from sklearn.linear_model import LogisticRegression --import_list from sklearn.neural_network import MLPClassifier --import_list from estimators.statistical_descriptor import * --import_list from estimators.band_transform import * --metrics_to_report f1_score_macro --metrics_to_report f1_score_weighted --metrics_to_report f1_score_multiclass --metrics_to_report accuracy_score --metrics_to_report precision_score_macro --metrics_to_report recall_score_macro --metrics_to_report roc_auc_score --metrics_to_report log_loss --metrics_to_report kappa_score --metrics_to_report confusion_matrix --seed 100150 --pipeline [['KNN_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['KNeighborsClassifier', {'n_neighbors': 50}]] --pipeline [['RandomForest_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['RandomForestClassifier', {'n_estimators': 200}, {'criterion': 'entropy'}]] --pipeline [['MLP_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['MLPClassifier', {'alpha': 0.01}]] --storage_path /home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/results/pipeline/run_26/group_0", "python evaluation/learning_models.py --data_path /home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5 --fold_method combinationFold --labeling_method crocus --balancing_method undersample --request (date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000)) --shuffle_data True --balance_data False --import_list from sklearn.svm import SVC --import_list from sklearn.neighbors import KNeighborsClassifier --import_list from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier --import_list from sklearn.linear_model import LogisticRegression --import_list from sklearn.neural_network import MLPClassifier --import_list from estimators.statistical_descriptor import * --import_list from estimators.band_transform import * --metrics_to_report f1_score_macro --metrics_to_report f1_score_weighted --metrics_to_report f1_score_multiclass --metrics_to_report accuracy_score --metrics_to_report precision_score_macro --metrics_to_report recall_score_macro --metrics_to_report roc_auc_score --metrics_to_report log_loss --metrics_to_report kappa_score --metrics_to_report confusion_matrix --seed 100150 --pipeline [['KNN_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['KNeighborsClassifier', {'n_neighbors': 50}]] --pipeline [['RandomForest_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['RandomForestClassifier', {'n_estimators': 200}, {'criterion': 'entropy'}]] --pipeline [['MLP_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['MLPClassifier', {'alpha': 0.01}]] --pipeline [['SVMrbf_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['SVC', {'kernel': 'rbf'}, {'probability': True}]] --pipeline [['LogisticR_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['LogisticRegression']] --pipeline [['AdaBoost_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['AdaBoostClassifier', {'n_estimators': 200}]] --storage_path /home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/results/pipeline/run_26/group_1"], 'commit_sha': '49de8221a85c92025f2382abf5134b3ee4a77c3c', 'end_time': [(2024, 7, 1, 13, 37, 16, 909068), (2024, 7, 1, 13, 41, 18, 717638)], 'executable': 'evaluation/learning_models.py', 'executable_command': 'python', 'experiment_id': 1, 'groups_of_parameters': [{'--balance_data': False, '--balancing_method': 'undersample', '--data_path': '/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5', '--fold_method': 'combinationFold', '--import_list': ['from sklearn.svm import SVC', 'from sklearn.neighbors import KNeighborsClassifier', 'from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier', 'from sklearn.linear_model import LogisticRegression', 'from sklearn.neural_network import MLPClassifier', 'from estimators.statistical_descriptor import *', 'from estimators.band_transform import *'], '--labeling_method': 'crocus', '--metrics_to_report': ['f1_score_macro', 'f1_score_weighted', 'f1_score_multiclass', 'accuracy_score', 'precision_score_macro', 'recall_score_macro', 'roc_auc_score', 'log_loss', 'kappa_score', 'confusion_matrix'], '--pipeline': [[['KNN_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['KNeighborsClassifier', {'n_neighbors': 50}]], [['RandomForest_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['RandomForestClassifier', {'n_estimators': 200}, {'criterion': 'entropy'}]], [['MLP_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['MLPClassifier', {'alpha': 0.01}]]], '--request': '(date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000))', '--seed': 100150, '--shuffle_data': True}, {'--balance_data': False, '--balancing_method': 'undersample', '--data_path': '/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5', '--fold_method': 'combinationFold', '--import_list': ['from sklearn.svm import SVC', 'from sklearn.neighbors import KNeighborsClassifier', 'from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier', 'from sklearn.linear_model import LogisticRegression', 'from sklearn.neural_network import MLPClassifier', 'from estimators.statistical_descriptor import *', 'from estimators.band_transform import *'], '--labeling_method': 'crocus', '--metrics_to_report': ['f1_score_macro', 'f1_score_weighted', 'f1_score_multiclass', 'accuracy_score', 'precision_score_macro', 'recall_score_macro', 'roc_auc_score', 'log_loss', 'kappa_score', 'confusion_matrix'], '--pipeline': [[['KNN_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['KNeighborsClassifier', {'n_neighbors': 50}]], [['RandomForest_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['RandomForestClassifier', {'n_estimators': 200}, {'criterion': 'entropy'}]], [['MLP_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['MLPClassifier', {'alpha': 0.01}]], [['SVMrbf_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['SVC', {'kernel': 'rbf'}, {'probability': True}]], [['LogisticR_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['LogisticRegression']], [['AdaBoost_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['AdaBoostClassifier', {'n_estimators': 200}]]], '--request': '(date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000))', '--seed': 100150, '--shuffle_data': True}], 'main_pid': 30846, 'pids': ['30860', '31348'], 'repertories': ['results/pipeline/run_26/group_0', 'results/pipeline/run_26/group_1'], 'run_id': 26, 'start_time': [(2024, 7, 1, 13, 34, 35, 443214), (2024, 7, 1, 13, 37, 16, 962683)], 'status': ['finished', 'finished'], 'storage_path': 'results/pipeline/run_26', 'working_directory': '/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline'}
x

{'commands': ["python evaluation/learning_models.py --data_path /home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5 --fold_method combinationFold --labeling_method crocus --balancing_method undersample --request (date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000)) --shuffle_data True --balance_data False --import_list from sklearn.svm import SVC --import_list from sklearn.neighbors import KNeighborsClassifier --import_list from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier --import_list from sklearn.linear_model import LogisticRegression --import_list from sklearn.neural_network import MLPClassifier --import_list from estimators.statistical_descriptor import * --import_list from estimators.band_transform import * --metrics_to_report f1_score_macro --metrics_to_report f1_score_weighted --metrics_to_report f1_score_multiclass --metrics_to_report 

In [ ]:
{'commands': ["python evaluation/learning_models.py --data_path /home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5 --fold_method combinationFold --labeling_method crocus --balancing_method undersample --request (date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000)) --shuffle_data True --balance_data False --import_list from sklearn.svm import SVC --import_list from sklearn.neighbors import KNeighborsClassifier --import_list from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier --import_list from sklearn.linear_model import LogisticRegression --import_list from sklearn.neural_network import MLPClassifier --import_list from estimators.statistical_descriptor import * --import_list from estimators.band_transform import * --metrics_to_report f1_score_macro --metrics_to_report f1_score_weighted --metrics_to_report f1_score_multiclass --metrics_to_report accuracy_score --metrics_to_report precision_score_macro --metrics_to_report recall_score_macro --metrics_to_report roc_auc_score --metrics_to_report log_loss --metrics_to_report kappa_score --metrics_to_report confusion_matrix --seed 100150 --pipeline [['KNN_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['KNeighborsClassifier', {'n_neighbors': 50}]] --pipeline [['RandomForest_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['RandomForestClassifier', {'n_estimators': 200}, {'criterion': 'entropy'}]] --pipeline [['MLP_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['MLPClassifier', {'alpha': 0.01}]] --storage_path /home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/results/pipeline/run_26/group_0",
  "python evaluation/learning_models.py --data_path /home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5 --fold_method combinationFold --labeling_method crocus --balancing_method undersample --request (date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000)) --shuffle_data True --balance_data False --import_list from sklearn.svm import SVC --import_list from sklearn.neighbors import KNeighborsClassifier --import_list from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier --import_list from sklearn.linear_model import LogisticRegression --import_list from sklearn.neural_network import MLPClassifier --import_list from estimators.statistical_descriptor import * --import_list from estimators.band_transform import * --metrics_to_report f1_score_macro --metrics_to_report f1_score_weighted --metrics_to_report f1_score_multiclass --metrics_to_report accuracy_score --metrics_to_report precision_score_macro --metrics_to_report recall_score_macro --metrics_to_report roc_auc_score --metrics_to_report log_loss --metrics_to_report kappa_score --metrics_to_report confusion_matrix --seed 100150 --pipeline [['KNN_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['KNeighborsClassifier', {'n_neighbors': 50}]] --pipeline [['RandomForest_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['RandomForestClassifier', {'n_estimators': 200}, {'criterion': 'entropy'}]] --pipeline [['MLP_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['MLPClassifier', {'alpha': 0.01}]] --pipeline [['SVMrbf_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['SVC', {'kernel': 'rbf'}, {'probability': True}]] --pipeline [['LogisticR_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['LogisticRegression']] --pipeline [['AdaBoost_direct'], ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}], ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}], ['Hist_SAR'], ['AdaBoostClassifier', {'n_estimators': 200}]] --storage_path /home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/results/pipeline/run_26/group_1"],
 'commit_sha': '49de8221a85c92025f2382abf5134b3ee4a77c3c',
 'end_time': [(2024, 7, 1, 13, 37, 16, 909068),
  (2024, 7, 1, 13, 41, 18, 717638)],
 'executable': 'evaluation/learning_models.py',
 'executable_command': 'python',
 'experiment_id': 1,
 'groups_of_parameters': [{'--balance_data': False,
   '--balancing_method': 'undersample',
   '--data_path': '/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5',
   '--fold_method': 'combinationFold',
   '--import_list': ['from sklearn.svm import SVC',
    'from sklearn.neighbors import KNeighborsClassifier',
    'from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier',
    'from sklearn.linear_model import LogisticRegression',
    'from sklearn.neural_network import MLPClassifier',
    'from estimators.statistical_descriptor import *',
    'from estimators.band_transform import *'],
   '--labeling_method': 'crocus',
   '--metrics_to_report': ['f1_score_macro',
    'f1_score_weighted',
    'f1_score_multiclass',
    'accuracy_score',
    'precision_score_macro',
    'recall_score_macro',
    'roc_auc_score',
    'log_loss',
    'kappa_score',
    'confusion_matrix'],
   '--pipeline': [[['KNN_direct'],
     ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}],
     ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}],
     ['Hist_SAR'],
     ['KNeighborsClassifier', {'n_neighbors': 50}]],
    [['RandomForest_direct'],
     ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}],
     ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}],
     ['Hist_SAR'],
     ['RandomForestClassifier',
      {'n_estimators': 200},
      {'criterion': 'entropy'}]],
    [['MLP_direct'],
     ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}],
     ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}],
     ['Hist_SAR'],
     ['MLPClassifier', {'alpha': 0.01}]]],
   '--request': '(date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000))',
   '--seed': 100150,
   '--shuffle_data': True},
  {'--balance_data': False,
   '--balancing_method': 'undersample',
   '--data_path': '/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5',
   '--fold_method': 'combinationFold',
   '--import_list': ['from sklearn.svm import SVC',
    'from sklearn.neighbors import KNeighborsClassifier',
    'from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier',
    'from sklearn.linear_model import LogisticRegression',
    'from sklearn.neural_network import MLPClassifier',
    'from estimators.statistical_descriptor import *',
    'from estimators.band_transform import *'],
   '--labeling_method': 'crocus',
   '--metrics_to_report': ['f1_score_macro',
    'f1_score_weighted',
    'f1_score_multiclass',
    'accuracy_score',
    'precision_score_macro',
    'recall_score_macro',
    'roc_auc_score',
    'log_loss',
    'kappa_score',
    'confusion_matrix'],
   '--pipeline': [[['KNN_direct'],
     ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}],
     ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}],
     ['Hist_SAR'],
     ['KNeighborsClassifier', {'n_neighbors': 50}]],
    [['RandomForest_direct'],
     ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}],
     ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}],
     ['Hist_SAR'],
     ['RandomForestClassifier',
      {'n_estimators': 200},
      {'criterion': 'entropy'}]],
    [['MLP_direct'],
     ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}],
     ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}],
     ['Hist_SAR'],
     ['MLPClassifier', {'alpha': 0.01}]],
    [['SVMrbf_direct'],
     ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}],
     ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}],
     ['Hist_SAR'],
     ['SVC', {'kernel': 'rbf'}, {'probability': True}]],
    [['LogisticR_direct'],
     ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}],
     ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}],
     ['Hist_SAR'],
     ['LogisticRegression']],
    [['AdaBoost_direct'],
     ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}],
     ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}],
     ['Hist_SAR'],
     ['AdaBoostClassifier', {'n_estimators': 200}]]],
   '--request': '(date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000))',
   '--seed': 100150,
   '--shuffle_data': True}],
 'main_pid': 30846,
 'pids': ['30860', '31348'],
 'repertories': ['results/pipeline/run_26/group_0',
  'results/pipeline/run_26/group_1'],
 'run_id': 26,
 'start_time': [(2024, 7, 1, 13, 34, 35, 443214),
  (2024, 7, 1, 13, 37, 16, 962683)],
 'status': ['finished', 'finished'],
 'storage_path': 'results/pipeline/run_26',
 'working_directory': '/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline'}

In [108]:
models = [x["groups_of_parameters"][0]["--pipeline"][i][0][0] for i in range(len(x["groups_of_parameters"][0]["--pipeline"]))] 
models


['KNN_direct', 'RandomForest_direct', 'MLP_direct']

In [109]:
y= ['./results/pipeline/run_26/group_1', './results/pipeline/run_26/group_0']


In [111]:
y.sort()
y

['./results/pipeline/run_26/group_0', './results/pipeline/run_26/group_1']